# 세션 6 – 다단계 파이프라인 (계획 → 실행 → 수정)

라우팅 함수를 사용하여 단계별로 모델을 선택한 후 최종 답변을 수정합니다.


### 설명: 종속성 설치
`foundry-local-sdk`와 `openai`를 설치하여 단계별 채팅 호출에 필요합니다. 다시 실행해도 안전합니다.


# 시나리오
다단계 모델 파이프라인: (1) 작업을 개별 단계로 계획, (2) 의도 기반 모델 선택을 통해 각 단계를 실행, (3) 최종 합성된 답변을 개선. 효율성을 유지하면서 품질을 보존하기 위해 이질적인 SLM 기능을 연결하는 방법을 보여줍니다.


In [13]:
!pip install -q foundry-local-sdk openai

### 설명: 핵심 임포트
의도 감지를 위한 regex, 별칭별 첨부를 위한 Foundry Local 매니저, 그리고 채팅 완료를 위한 OpenAI 클라이언트를 임포트합니다.


In [14]:
import re
from foundry_local import FoundryLocalManager
from openai import OpenAI

### 설명: 기능 카탈로그 및 규칙
기능 인식 카탈로그와 정규식 규칙을 정의하여 단계 텍스트를 의도 카테고리(코드, 요약, 분류, 일반)로 매핑합니다. 여러 모델이 동일한 의도를 지원하는 경우 우선순위 값이 낮은 것이 우선됩니다.


In [15]:
CATALOG = {
 'phi-4-mini': {'capabilities':['general','summarize'],'priority':2},
 'qwen2.5-coder-7b': {'capabilities':['code','refactor'],'priority':1},
 'qwen2.5-0.5b': {'capabilities':['classification','fast'],'priority':3},
}
RULES = [
 (re.compile('code|refactor|function', re.I), 'code'),
 (re.compile('summari|abstract|tl;dr', re.I), 'summarize'),
 (re.compile('classif|category|label', re.I), 'classification'),
]

### 설명: 의도, 모델 선택 및 채팅 도우미
제공:
- `detect_intent`: 정규 표현식을 기반으로 한 분류.
- `pick_model`: 기능과 우선순위에 따라 최적의 별칭 선택.
- `chat`: 별칭별 클라이언트를 생성하고 단일 턴 응답을 반환하는 편리한 래퍼.


In [16]:
def detect_intent(text: str):
    """Return an intent label based on simple regex rules; falls back to 'general'."""
    for pat, intent in RULES:
        if pat.search(text):
            return intent
    return 'general'

def pick_model(intent: str) -> str:
    """Pick the best model for an intent using capability match first, then priority."""
    ranked = []
    for name, meta in CATALOG.items():
        ranked.append((name, intent in meta['capabilities'], meta['priority']))
    # Sort: capability match (True first), then lower priority value
    ranked.sort(key=lambda t: (not t[1], t[2]))
    return ranked[0][0]

def chat(alias: str, content: str, temp: float = 0.4) -> str:
    """Simple helper to send a single user message to a Foundry Local model via OpenAI client."""
    m = FoundryLocalManager(alias)
    client = OpenAI(base_url=m.endpoint, api_key=m.api_key or 'not-needed')
    mid = m.get_model_info(alias).id
    resp = client.chat.completions.create(
        model=mid,
        messages=[{'role': 'user', 'content': content}],
        max_tokens=180,
        temperature=temp,
    )
    return resp.choices[0].message.content

### 설명: 다단계 파이프라인 함수
파이프라인을 구현합니다: 계획 → 단계 분석 → 의도 기반 라우팅으로 각각 실행 → 결합된 출력 정제. 검토 또는 평가를 위한 구조화된 dict를 반환합니다.


In [17]:
def pipeline(task: str):
    """Multi-step pipeline: plan → execute steps → refine final answer.

    Returns dict with keys: plan (raw plan text), steps (list of tuples), final (refined answer).
    Each step tuple: (index, step_text, step_result, model_alias_used)
    """
    # 1. Plan
    plan_alias = pick_model('general')
    plan_prompt = (
        "Break the task into 3 concise, actionable steps (no extra commentary).\n"
        f"Task: {task}"
    )
    plan = chat(plan_alias, plan_prompt)

    # 2. Parse steps (robust to numbering or bullet styles)
    raw_lines = [l.strip() for l in plan.splitlines() if l.strip()]
    steps = []
    for line in raw_lines:
        cleaned = re.sub(r'^\d+[).:-]?\s*', '', line)  # remove leading numbering
        cleaned = re.sub(r'^[-*]\s*', '', cleaned)      # remove bullet markers
        if cleaned:
            steps.append(cleaned)
        if len(steps) == 3:
            break
    if not steps:
        steps = [task]

    # 3. Execute steps
    outputs = []
    for idx, step in enumerate(steps, 1):
        intent = detect_intent(step)
        exec_alias = pick_model(intent)
        exec_prompt = (
            f"Execute step {idx} for the overall task.\n"
            f"Overall task: {task}\n"
            f"Step {idx}: {step}\n"
            "Return a concise result focusing only on the step objective."
        )
        result = chat(exec_alias, exec_prompt)
        outputs.append((idx, step, result, exec_alias))

    # 4. Refine final answer
    refine_alias = pick_model('summarize')
    combined = "\n\n".join(
        f"Step {idx} ({alias}) Output:\n{res}" for idx, _step, res, alias in outputs
    )
    refine_prompt = (
        "You are a senior assistant. Synthesize these step outputs into a cohesive final answer.\n"
        "Ensure clarity, avoid repetition, and highlight improvements or key insights if relevant.\n\n"
        f"Task: {task}\n\n"
        f"Step Outputs:\n{combined}"
    )
    final_ans = chat(refine_alias, refine_prompt)

    return {"plan": plan, "steps": outputs, "final": final_ans}

### 설명: 예제 작업 실행
리팩터링 중심의 작업에서 혼합된 의도(코드 + 요약)를 보여주는 전체 파이프라인을 실행합니다. 결과 딕셔너리는 원시 계획, 단계별 출력(선택된 모델 별칭 포함), 그리고 최종적으로 합성된 답변을 보여줍니다.


In [18]:
result = pipeline('Generate a refactored version of a slow Python loop and summarize performance gains.')
result

{'plan': '1. Profile the existing slow Python loop to identify bottlenecks.\n2. Refactor the loop using optimized techniques (e.g., list comprehensions, map, or itertools).\n3. Compare the performance of the refactored loop with the original using a benchmarking tool and summarize the gains.',
 'steps': [(1,
   'Profile the existing slow Python loop to identify bottlenecks.',
   "To execute step 1, you would use a profiling tool like `cProfile` in Python to analyze the performance of the existing slow loop. Here's an example of how you might do this:\n\n```python\nimport cProfile\n\ndef slow_loop():\n    # Example of a slow loop\n    result = []\n    for i in range(1000000):\n        result.append(i * i)\n    return result\n\n# Profile the slow loop\ncProfile.run('slow_loop()', 'profile_stats')\n\n# To see the results, you can use pstats module\nimport pstats\np = pstats.Stats('profile_stats')\np.sort_stats('cumulative').print_stats()\n```\n\nThis code will run the `slow_loop` function

### 설명: 결과 객체 표시
구조화된 파이프라인 출력을 보여주어 빠른 검사나 후속 평가(예: 단계 품질 측정 또는 개선 효과 평가)에 활용할 수 있습니다.



---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있으나, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서의 원어 버전이 권위 있는 출처로 간주되어야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 당사는 책임을 지지 않습니다.
